In [1]:
import os
import shutil

def collect_cg_pdfs(source_root, destination_root):
    """
    Walk through all subfolders of 'source_root' and copy any files that:
    - Start with 'CG. ' (case-sensitive)
    - End with '.pdf' (case-insensitive)
    into the 'destination_root' folder.
    """

    # Ensure destination exists
    os.makedirs(destination_root, exist_ok=True)

    for folder_path, _, files in os.walk(source_root):
        for file_name in files:
            # Match files starting with "CG. " and ending with ".pdf"
            if file_name.startswith("CG. ") and file_name.lower().endswith(".pdf"):
                source_path = os.path.join(folder_path, file_name)
                dest_path = os.path.join(destination_root, file_name)

                # Handle duplicate filenames by appending a counter
                base, ext = os.path.splitext(file_name)
                counter = 1
                while os.path.exists(dest_path):
                    dest_path = os.path.join(destination_root, f"{base}_{counter}{ext}")
                    counter += 1

                try:
                    shutil.copy2(source_path, dest_path)
                    print(f"Copied: {source_path} -> {dest_path}")
                except Exception as e:
                    print(f"Error copying {source_path}: {e}")

if __name__ == "__main__":
    # Example usage:
    source_directory = r"/Users/fahad/Desktop/car_stolen/Cars/"       # 🔹 Change this
    destination_directory = r"/Users/fahad/Desktop/car_stolen/PDFS/"    # 🔹 Change this
    collect_cg_pdfs(source_directory, destination_directory)


Copied: /Users/fahad/Desktop/car_stolen/Cars/VOLKSWAGEN/E-CRAFTER/GE/GE-295-TV/CG. GE-295-TV.pdf -> /Users/fahad/Desktop/car_stolen/PDFS/CG. GE-295-TV.pdf
Copied: /Users/fahad/Desktop/car_stolen/Cars/VOLKSWAGEN/E-CRAFTER/GE/GE-122-JH/CG. GE-122-JH.pdf -> /Users/fahad/Desktop/car_stolen/PDFS/CG. GE-122-JH.pdf
Copied: /Users/fahad/Desktop/car_stolen/Cars/VOLKSWAGEN/ID7/HB/HB-592-SH/CG. HB-592-SH.pdf -> /Users/fahad/Desktop/car_stolen/PDFS/CG. HB-592-SH.pdf
Copied: /Users/fahad/Desktop/car_stolen/Cars/VOLKSWAGEN/ID7/HB/HB-159-MP/CG. HB-159-MP.pdf -> /Users/fahad/Desktop/car_stolen/PDFS/CG. HB-159-MP.pdf
Copied: /Users/fahad/Desktop/car_stolen/Cars/VOLKSWAGEN/ID7/HB/HB-880-SW/CG. HB-880-SW.pdf -> /Users/fahad/Desktop/car_stolen/PDFS/CG. HB-880-SW.pdf
Copied: /Users/fahad/Desktop/car_stolen/Cars/VOLKSWAGEN/ID7/HB/HB-825-VG/CG. HB-825-VG.pdf -> /Users/fahad/Desktop/car_stolen/PDFS/CG. HB-825-VG.pdf
Copied: /Users/fahad/Desktop/car_stolen/Cars/VOLKSWAGEN/ID7/HB/HB-835-SW/CG. HB-835-SW.pdf -> 

In [2]:
import pandas as pd

def filter_car_models(input_excel, output_excel):
    """
    Reads an Excel sheet, selects rows where 'Modèle' column contains
    specific target car models, and saves them into a new Excel file.
    """

    # ✅ Define target models
    target_models = [
        "MODEL 3",
        "MODEL Y",
        "ENYAQ",
        "C-HR",
        "ID7"
    ]

    # Load the Excel file
    df = pd.read_excel(input_excel)

    # Check if 'Modèle' column exists
    if 'Modèle' not in df.columns:
        raise ValueError("Column 'Modèle' not found in the Excel file.")

    # Convert 'Modèle' to string and uppercase for robust comparison
    df['Modèle'] = df['Modèle'].astype(str).str.upper().str.strip()

    # Filter rows that CONTAIN any of the target model names (partial match)
    mask = df['Modèle'].apply(
        lambda x: any(model in x for model in target_models)
    )
    filtered_df = df[mask]

    # Save the filtered data to a new Excel file
    filtered_df.to_excel(output_excel, index=False)

    print(f"✅ Filtered {len(filtered_df)} rows and saved to: {output_excel}")


if __name__ == "__main__":
    # Example usage:
    input_file = r"/Users/fahad/Desktop/car_stolen/export_vehicle_20251016_1634.xlsx"       # 🔹 Change this
    output_file = r"/Users/fahad/Desktop/car_stolen/filtered_vehicles.xlsx"  # 🔹 Change this
    filter_car_models(input_file, output_file)


✅ Filtered 850 rows and saved to: /Users/fahad/Desktop/car_stolen/filtered_vehicles.xlsx


## Date modified

In [3]:
import pandas as pd

def format_dates(input_excel, output_excel, date_column):
    """
    Reads an Excel file, reformats dates in a given column to DD/MM/YYYY,
    and removes time information like '00:00:00'.
    """

    # Load the Excel file
    df = pd.read_excel(input_excel)

    # Check if the column exists
    if date_column not in df.columns:
        raise ValueError(f"Column '{date_column}' not found in Excel file.")

    # Convert to datetime (handles text or mixed formats gracefully)
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')

    # Drop rows with invalid or missing dates (optional)
    # df = df.dropna(subset=[date_column])

    # Reformat date to DD/MM/YYYY (string)
    df[date_column] = df[date_column].dt.strftime('%d/%m/%Y')

    # Save to a new Excel file
    df.to_excel(output_excel, index=False)

    print(f"✅ Dates in '{date_column}' reformatted and saved to: {output_excel}")


if __name__ == "__main__":
    # Example usage:
    input_file = r"/Users/fahad/Desktop/car_stolen/filtered_vehicles.xlsx"       # 🔹 change this
    output_file = r"/Users/fahad/Desktop/car_stolen/filtered_vehicles_date.xlsx"  # 🔹 change this
    column_name = "Date de première immatriculation"      # 🔹 change this

    format_dates(input_file, output_file, column_name)


✅ Dates in 'Date de première immatriculation' reformatted and saved to: /Users/fahad/Desktop/car_stolen/filtered_vehicles_date.xlsx


## Check column Financeur

In [4]:
import pandas as pd

def remove_missing_financeur(input_excel, output_excel):
    """
    Reads an Excel file, removes rows where the 'Financeur' column
    is missing or empty, and saves the cleaned data to a new Excel file.
    """

    # Load Excel file
    df = pd.read_excel(input_excel)

    # Check if column exists
    if 'Financeur' not in df.columns:
        raise ValueError("Column 'Financeur' not found in the Excel file.")

    # Remove rows where 'Financeur' is missing or empty
    cleaned_df = df[df['Financeur'].notna() & (df['Financeur'].astype(str).str.strip() != '')]

    # Save to a new Excel file
    cleaned_df.to_excel(output_excel, index=False)

    print(f"✅ Removed {len(df) - len(cleaned_df)} rows with missing 'Financeur' values.")
    print(f"💾 Cleaned file saved to: {output_excel}")


if __name__ == "__main__":
    # Example usage
    input_file = r"/Users/fahad/Desktop/car_stolen/filtered_vehicles_date.xlsx"           # 🔹 Change this
    output_file = r"/Users/fahad/Desktop/car_stolen/filtered_vehicles_date_F.xlsx"       # 🔹 Change this

    remove_missing_financeur(input_file, output_file)


✅ Removed 102 rows with missing 'Financeur' values.
💾 Cleaned file saved to: /Users/fahad/Desktop/car_stolen/filtered_vehicles_date_F.xlsx


In [5]:
import pandas as pd

def map_and_copy_rows(source_excel, destination_excel):
    """
    Reads data from a source Excel file, selects and renames columns
    according to an exact header mapping, and saves to a new Excel file
    with headers preserved exactly as defined.
    """

    # Load source Excel file
    df = pd.read_excel(source_excel)

    # ✅ Define exact mapping from source columns → destination headers
    column_mapping = {
        'Immatriculation': "Numéro d'immatriculation",
        'Constructeur': "Constructeur",
        'Modèle': "Modèle",
        'Date de première immatriculation': [
            "Date de première immatriculation du véhicule",
            "Date du certificat d'immatriculation"
        ],
        'Financeur': "ou (Si personne morale) Raison sociale"
    }

    # ✅ Check all required columns exist in source file
    missing = [col for col in column_mapping if col not in df.columns]
    if missing:
        raise ValueError(f"❌ Missing columns in source file: {missing}")

    # ✅ Create a new DataFrame with exact destination headers
    new_df = pd.DataFrame({
        column_mapping['Immatriculation']: df['Immatriculation'],
        column_mapping['Constructeur']: df['Constructeur'],
        column_mapping['Modèle']: df['Modèle'],
        column_mapping['Date de première immatriculation'][0]: df['Date de première immatriculation'],
        column_mapping['Date de première immatriculation'][1]: df['Date de première immatriculation'],
        column_mapping['Financeur']: df['Financeur']
    })

    # ✅ Save to Excel with headers EXACTLY as defined
    new_df.to_excel(destination_excel, index=False, header=True)

    print(f"✅ Successfully created file: {destination_excel}")
    print(f"📊 Total rows copied: {len(new_df)}")


if __name__ == "__main__":
    # Example usage
    source_file = r"/Users/fahad/Desktop/car_stolen/filtered_vehicles_date_F.xlsx"          # 🔹 Change this
    destination_file = r"/Users/fahad/Desktop/car_stolen/entries.xlsx"   # 🔹 Change this

    map_and_copy_rows(source_file, destination_file)


✅ Successfully created file: /Users/fahad/Desktop/car_stolen/entries.xlsx
📊 Total rows copied: 748


In [3]:
import os
import pandas as pd

folder_path = "/Users/fahad/Desktop/car_stolen/PDFs"  # change this to your folder path

# Collect all PDF file names
pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]

# Clean file names: remove "CG. " at start and ".pdf" at end
cleaned_names = []
for f in pdf_files:
    name = f
    if name.lower().endswith(".pdf"):
        name = name[:-4]  # remove .pdf
    if name.startswith("CG. "):
        name = name.replace("CG. ", "", 1)  # remove prefix once
    cleaned_names.append(name.strip())

# Save to Excel
df = pd.DataFrame(cleaned_names, columns=["file_name"])
df.to_excel("all_pdf_files_cleaned.xlsx", index=False)

print(f"✅ Saved {len(cleaned_names)} cleaned file names to all_pdf_files_cleaned.xlsx")

✅ Saved 1778 cleaned file names to all_pdf_files_cleaned.xlsx


In [5]:
import os
import pandas as pd

# --- Folder paths ---
pdf_folder = "/Users/fahad/Desktop/car_stolen/PDFs"        # Folder containing PDFs
txt_folder = "/Users/fahad/Desktop/car_stolen/textsVision"     # Folder containing text files

# --- Helper function to clean filenames ---
def clean_name(filename, prefix="CG. ", suffix=".pdf"):
    name = filename
    if name.lower().endswith(suffix):
        name = name[:-len(suffix)]
    if name.startswith(prefix):
        name = name.replace(prefix, "", 1)
    return name.strip()

# --- Collect and clean PDF names ---
pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]
pdf_names = [clean_name(f, suffix=".pdf") for f in pdf_files]

# --- Collect and clean TXT names ---
txt_files = [f for f in os.listdir(txt_folder) if f.lower().endswith(".txt")]
txt_names = [clean_name(f, suffix=".txt") for f in txt_files]

# --- Merge into DataFrame ---
pdf_df = pd.DataFrame(pdf_names, columns=["pdf_name"])
txt_df = pd.DataFrame(txt_names, columns=["txt_name"])

# Outer join on matching names (to include all)
merged_df = pd.merge(pdf_df, txt_df, how="outer", left_on="pdf_name", right_on="txt_name")

# Save to Excel
merged_df.to_excel("matched_files.xlsx", index=False)

print(f"✅ Saved {len(merged_df)} matched filenames to matched_files.xlsx")


✅ Saved 1778 matched filenames to matched_files.xlsx


In [15]:
import os
import re
import pandas as pd

# === CONFIGURATION ===
excel_path    = "filtered_vehicles_date_F.xlsx"      # your Excel (input/output)
text_folder   = "/Users/fahad/Desktop/car_stolen/textsVision"
missing_excel = "missing_excel.xlsx"                 # report for blanks/missing

# Keep EXACT capitalization as canonical labels
KNOWN_FINANCIERS = [
    "VOLKSWAGEN BANK GESELLSCHAFT MIT BESCHRAENKTER HAFTUNG",
    "CREDIT MUTUEL LEASING",
    "TOYOTA FRANCE FINANCEMENT ET TFRF",
    "TOYOTA FRANCE FINANCEMENT ET TERF",
    "VOLKSWAGEN FINANCIAL SERVICES - VOLKSWAGEN BANK",
    "VOLKSWAGEN FINANCIAL SERVICES",
    "VOLKSWAGEN BANK",
    "GCA LEASE",
    "TOYOTA KREDITBANK GMBH",
    "OLINN SERVICES"
]

# === DEFINITE NON-FINANCIER PHRASES (skip if found) ===
BLOCKLIST = [
    "CERTIFICAT D IMMATRICULATION",
    "CERTIFICAT D'IMMATRICULATION",
    "DATE DE 1 RE IMMATRICULATION",
    "DATE DE 1ERE IMMATRICULATION",
    "N° IMMATRICULATION",
    "N IMMAT",
    "IMMAT",
    "A.",
    "B.",
    "EST LE PROPRIETAIRE DU VEHICULE",
    "C.4A",
    "C4A",
    "C.3",
    "C3",
    "C.4.1",
    "C41",
]

# ---------- helpers ----------
def norm_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def tokenized_whitespace_regex(label: str) -> re.Pattern:
    """Match label ignoring newlines/extra spaces."""
    tokens = [re.escape(t) for t in re.split(r"\s+", label.strip()) if t]
    pattern = r"\b" + r"\s+".join(tokens) + r"\b"
    return re.compile(pattern, flags=re.IGNORECASE)

# Precompile patterns for speed
KNOWN_PATTERNS = [(label, tokenized_whitespace_regex(label)) for label in KNOWN_FINANCIERS]

def get_c1_block(upper_text: str) -> str | None:
    """Extract ~400 characters following C1 / C.1 (usually contains financier)."""
    m = re.search(r"(C\.?\s*1)\s*[:\-]?\s*([\s\S]{0,400})", upper_text)
    if not m:
        return None
    window = m.group(2)

    stop = re.search(r"\n?\s*C\s*[\.\s]?\s*[2345]\b|\n?\s*D\s*[\.\s]?\s*\d\b", window)
    if stop:
        window = window[:stop.start()]

    lines = [norm_spaces(l) for l in window.splitlines() if norm_spaces(l)]
    if not lines:
        return None

    candidate = norm_spaces(" ".join(lines[:2]))
    candidate = re.split(r"\bC\s*[\.\s]?\s*[2345]\b|\bD\s*[\.\s]?\s*\d\b", candidate)[0]
    candidate = re.sub(r"[^A-Z0-9 ,.'&/()\-]", " ", candidate)
    candidate = norm_spaces(candidate)
    return candidate if candidate else None

def pick_financier(upper_text: str) -> str | None:
    """
    1) Return canonical financier name if exact match (ignore surroundings).
    2) Else, return cleaned C1 text (if not in blocklist).
    3) Else, None.
    """
    c1_block = get_c1_block(upper_text)
    if not c1_block:
        return None

    # Step 1: known canonical match
    for canonical, pat in KNOWN_PATTERNS:
        if pat.search(c1_block):
            return canonical  # exact known match only

    # Step 2: fallback to full name, but ignore junk/blocklist
    for bad in BLOCKLIST:
        if bad in c1_block:
            return None

    # must be meaningful (at least 2 alphabetic words)
    if len(re.findall(r"[A-Z]{3,}", c1_block)) < 2:
        return None

    return c1_block


# ---------- main ----------
df = pd.read_excel(excel_path)
financiers = []
missing_rows = []

for _, row in df.iterrows():
    immat = str(row["Immatriculation"]).strip().upper()
    txt_name = f"CG. {immat}.txt"
    txt_path = os.path.join(text_folder, txt_name)

    if not os.path.exists(txt_path):
        financiers.append("")
        missing_rows.append({"Immatriculation": immat, "Reason": "File missing"})
        continue

    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()

    upper_text = raw.upper()
    chosen = pick_financier(upper_text)

    if chosen:
        financiers.append(chosen)
    else:
        financiers.append("")
        missing_rows.append({"Immatriculation": immat, "Reason": "No valid financier found"})

# Write results to same Excel
df["Financier"] = financiers
df.to_excel(excel_path, index=False)
print(f"✅ Updated {excel_path} with 'Financier'.")

# Write missing report
if missing_rows:
    pd.DataFrame(missing_rows).to_excel(missing_excel, index=False)
    print(f"⚠️ {len(missing_rows)} rows saved to {missing_excel}")
else:
    print("🎉 All entries processed successfully.")


✅ Updated filtered_vehicles_date_F.xlsx with 'Financier'.
⚠️ 362 rows saved to missing_excel.xlsx
